## Making CoCo file from json file

In [7]:
import json

def json_to_coco(json_data):
    coco_annotations = {
        'info': {},
        'licenses': [],
        'categories': [],
        'images': [],
        'annotations': []
    }

    category_mapping = {}
    category_id = 0
    annotation_id = 1

    images = json_data['_via_img_metadata']

    for image_id, image_data in images.items():
        filename = image_data['filename']
        width = image_data['size']
        height = image_data['size']
        annotations = image_data['regions']
        
        # Extract the numeric part of the image ID
        image_id = int(image_id.split('.')[0])
        
        # Add image information
        image_info = {
            'id': image_id,
            'file_name': filename,
            'width': width,
            'height': height,
        }
        coco_annotations['images'].append(image_info)

        for annotation in annotations:
            shape_attributes = annotation['shape_attributes']
            region_attributes = annotation['region_attributes']
            
            label = region_attributes['name']
            
            # Check if category is already defined, otherwise add it
            if label not in category_mapping:
                category_id += 1
                category_mapping[label] = category_id
                category_info = {
                    'id': category_id,
                    'name': label,
                    'supercategory': 'object',
                }
                coco_annotations['categories'].append(category_info)
            
            x = shape_attributes['x']
            y = shape_attributes['y']
            w = shape_attributes['width']
            h = shape_attributes['height']
            
            # Add annotation information
            annotation_info = {
                'id': annotation_id,
                'image_id': image_id,
                'category_id': category_mapping[label],
                'bbox': [x, y, w, h],
                'area': w * h,
                'iscrowd': 0,
            }
            coco_annotations['annotations'].append(annotation_info)
            
            annotation_id += 1

    return coco_annotations

# Load JSON annotation file
with open('C:/Users/Faisal/Downloads/Evaluation_Task/cat_dog_annotation.json', 'r') as file:
    json_data = json.load(file)

# Convert JSON annotation to COCO format
coco_annotations = json_to_coco(json_data)

# Save COCO annotations to a file
with open('C:/Users/Faisal/Downloads/Evaluation_Task/coco.json', 'w') as file:
    json.dump(coco_annotations, file)


## Augmentation

In [20]:
import os
import json
import random
from PIL import Image, ImageEnhance, ImageOps

def process_images(input_folder, output_folder):
    # Load JSON annotations
    with open(os.path.join(input_folder, 'C:/Users/Faisal/Downloads/Evaluation_Task/cat_dog_annotation.json'), 'r') as file:
        json_data = json.load(file)
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Iterate over images
    for image_id, image_data in json_data['_via_img_metadata'].items():
        filename = image_data['filename']
        
        # Check if the 'size' field is a dictionary or an integer
        if isinstance(image_data['size'], dict):
            width = image_data['size']['width']
            height = image_data['size']['height']
        else:
            image_path = os.path.join(input_folder, filename)
            with Image.open(image_path) as img:
                width, height = img.size
        
        annotations = image_data['regions']
        
        # Load original image
        original_image = Image.open(os.path.join(input_folder, filename))
        
        # Process the image for different angles
        for angle in range(0, 360, 5):
            # Step 1: Resize image
            resized_width = random.randint(int(width * 0.6), width)
            resized_height = random.randint(int(height * 0.6), height)
            resized_image, resized_annotations = resize_image(original_image, resized_width, resized_height, annotations)
            
            # Step 2: Add random brightness
            brightened_image = add_brightness(resized_image)
            
            # Step 3: Rotate image
            rotated_image, rotated_annotations = rotate_image(brightened_image, angle, resized_annotations)
            
            # Step 4: Save resulting image and annotations
            save_image_and_annotations(rotated_image, rotated_annotations, output_folder, image_id, angle)
    
    print('Image processing completed.')


def resize_image(image, width, height, annotations):
    width_ratio = width / image.width
    height_ratio = height / image.height
    
    # Resize image
    resized_image = image.resize((width, height))
    
    # Adjust bounding boxes
    for annotation in annotations:
        bbox = annotation['shape_attributes']
        bbox['x'] *= width_ratio
        bbox['y'] *= height_ratio
        bbox['width'] *= width_ratio
        bbox['height'] *= height_ratio
    
    return resized_image, annotations


def add_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    brightness_factor = random.uniform(0.8, 1.2)
    brightened_image = enhancer.enhance(brightness_factor)
    
    return brightened_image


def rotate_image(image, angle, annotations):
    rotated_image = image.rotate(angle, expand=True)
    
    # Adjust bounding boxes
    for annotation in annotations:
        bbox = annotation['shape_attributes']
        rotated_bbox = rotate_bbox(bbox, image.width, image.height, angle)
        annotation['shape_attributes'] = rotated_bbox
    
    return rotated_image, annotations


def rotate_bbox(bbox, image_width, image_height, angle):
    # Convert to radians
    angle_rad = math.radians(angle)
    
    # Center coordinates
    center_x = bbox['x'] + bbox['width'] / 2
    center_y = bbox['y'] + bbox['height'] / 2
    
    # New center coordinates after rotation
    new_center_x = round(center_x * math.cos(angle_rad) - center_y * math.sin(angle_rad))
    new_center_y = round(center_x * math.sin(angle_rad) + center_y * math.cos(angle_rad))
    
    # Update bbox coordinates
    bbox['x'] = new_center_x - bbox['width'] / 2
    bbox['y'] = new_center_y - bbox['height'] / 2
    
    # Handle out-of-bounds coordinates
    bbox['x'] = max(0, bbox['x'])
    bbox['y'] = max(0, bbox['y'])
    bbox['x'] = min(image_width - bbox['width'], bbox['x'])
    bbox['y'] = min(image_height - bbox['height'], bbox['y'])
    
    return bbox


def save_image_and_annotations(image, annotations, output_folder, image_id, angle):
    # Save image
    image_filename = f"{image_id}_angle_{angle}.jpeg"
    image.save(os.path.join(output_folder, image_filename))
    
    # Update image filename in annotations
    for annotation in annotations:
        annotation['filename'] = image_filename
    
    # Save annotations
    json_data = {
        '_via_img_metadata': {
            image_id: {
                'filename': image_filename,
                'regions': annotations
            }
        }
    }
    
    annotation_filename = f"{image_id}_angle_{angle}.json"
    with open(os.path.join(output_folder, annotation_filename), 'w') as file:
        json.dump(json_data, file)
    

# Set input and output folders
input_folder = 'C:/Users/Faisal/Downloads/Evaluation_Task/cat_dog'
output_folder = 'C:/Users/Faisal/Downloads/Evaluation_Task/dest'

# Process the images and generate new images with annotations
process_images(input_folder, output_folder)


Image processing completed.
